# Easing the Scorecard Data Download

- _Author: [Jan Hynek](jan.hynek@homecredit.eu)_  
- _TESTED ON:_
   - India
   - Indonesia

# Vision:
 __Make the data download as seamless (and thus easy) as possible.__
# Mission:
- Connect Python with Oracle SQL database.
- Create tables from the __vector__ master table in several Python commands. 
- Provide a fast way to download them.

# Explanation:
__DataCreator__ is the class that orchestrates the data creation (duuh).

The old way to download the data was to use SQL scripts, which are more or less standardised.  You needed to change dates on several places, change the data families, change the dates from-to, for which you need to download.

However, these scripts are run several times, with some small, slight changes.  
And all these small changes had to be inputted by hand, and it was easy to miss all the spots, where all the changes should be. 


# Requirements
 - Python:
     ```
     pandas
     sqlalchemy
     xlrd
     cx_Oracle
     ```
 - Oracle SQL __64-bit__ Instant Client
     - I know, I know. It is a pain.
     - However most of our infrastructure is running Oracle.
     - And this step needs to be solved only once.
  
     


<!-- 

# Easter Egg:
![](https://media1.tenor.com/images/6987362b140a83a2cba19942a48ddb20/tenor.gif?itemid=9258253)

-->


<details><summary>
    
## Install Oracle SQL __64-bit__ Instant Client:
(click to expand)
</summary>
    
  - [Download from here.](https://www.oracle.com/database/technologies/instant-client/winx64-64-downloads.html)
  - TESTED: 11.2 and 19.3 works on WIN10. 19.3 does not need oracle registration.
  - TESTED: ONLY 11.2 works on WIN7.
  - __Installation:__
    - Unzip folder on some convenient place, _which you will not change in near future_
    - Open start, and look up 'Environment Variables' Click on Environment Variables.
    - If possible, edit system variable 'Path' (requires admin account). 
    - If not, edit user variable 'Path'.
    - __IMPORTANT__: DO NOT DELETE ANYTHING FROM THIS 'Path' VARIABLE. APPEND ONLY.
    - Add to the 'Path' the folder where you unzipped the Oracle SQL Instant client. Separate it using semicolon.
    - to test the installation: open commandline (start -> cmd)
    - run `sqlplus`
    - you should see something like this:
    
```
SQL*Plus: Release 11.2.0.2.0 Production on Wed Apr 15 13:56:54 2020

Copyright (c) 1982, 2010, Oracle.  All rights reserved.
```
</details>

<details><summary>
    
# Quick run
(click to expand)

</summary>
    
- modify `config/db_config.cfg` with your info
   - input your username, password, database name
- modify `config/script_config.cfg
   - input base table (which does have column `SKP_CREDIT_CASE`)
   - input from/to dates
- run following:
    ```
    dc = DataCreator(
        db_config=r"config\db_config_template.cfg",
        script_config=r"config\script_config.cfg",
        log_level=10
    )
    dc.create_attributes()
    ```
  and get all attributes in `attributes.xlsx`.
- but wait, there is more - just follow the code here.
    
</details>

<details><summary>

# __DataCreator__ structure & documentation
(click to expand)

</summary>

## Templates
It consists from `templates`. These templates are the original SQL scripts, with _placeholders_. You can find them in the folder of the same name.

## Configs
However, __IMPORTANT__ are configs.

There are two mandatory configs.  
One is optional, if you have [tnsnames.ora](http://oradmin.homecredit.net/tnsnames.ora) correctly specified in your PC.  
All of them can be found in folder `configs`.

___

### __DATABASE CONFIG__: `config/db_config.cfg` <a class="anchor" id="db-config"></a>

#### Example:
```
[db]
user = MY_USERNAME[AP_UWI]
pw = mY_sECRET_pASSWORD
database = HDWIN.HOMECREDIT.IN
schema = ap_uwi
```

#### Individual config elements:
- `user`
    - the user name used for logging in the database.
    - can have the prespecified schema as well.
- `pw`
    - your database password.
- `database`
    - if you have `tnsnames.ora` correctly specified:
        - just input the name of the database.
    - if you don't:
        - either install `tnsnames.ora` from here: [tnsnames.ora](http://oradmin.homecredit.net/tnsnames.ora)
            - and ask EmbedIT for help
        - or [add a new connection string](#new-connection-string-addition).

 - `schema`
     - schema where should all final tables be stored. 
     - usually `ap_risk`, `ap_uwi`

___


### __CONNECTION STRINGS__: `config/connection_strings.py`
 - There should be no need to modify the connection strings, as they are downloaded automatically, from internal page.
 - optional with correctly specified `tnsnames.ora`
 - see README.md for further details
___

### __SCRIPT CONFIG__: `config/script_config.cfg` <a class="anchor" id="script-config"></a>
 
#### Example:
     
```
[predefined]
base = ap_uwi.jh_super_new_scorecard_base
attribute_selection_date_decision_start = 2020-01-15
attribute_selection_date_decision_end = 2020-01-30
prefix = qx_

[dataset]
date_decision_start = 2020-01-15
date_decision_end = 2020-01-30

[definitions]
families = {'static': 'vector_depth == 0',
    'prevAppl': '(path_0 == "applicantData") & path_1 == "previousApplicationDetails[]"',
    'credit': 'path_0 == "credit"',
    'other': 'family.isna()'}
name_replacements = {"\[]": "",
    "\.": "_",
    "address": "addr",
    "utility": "util",
    "employment": "empl",
    "registered": "regist",
    "payment": "pmt",
    }
```
#### Individual config elements:
- __predefined__
    - `base`
        - name of the base table, which defines for which credit cases will data be downloaded.
        - must contain following columns
            - `SKP_CREDIT_CASE`
            - `DATE_DECISION`
    - `attribute_selection_date_decision_start`
        - date limiting the dataset for the attribute list - start
    - `attribute_selection_date_decision_end`
        - date limiting the dataset for the attribute list - end
        - NOTE
            - this is to define such attributes which are currently used in the vector.
            - one month of data is usually enough
    - `prefix`
        - prefix to be used in the names of individual SQL tables.
        - should be initials.
- __dataset__
    - `date_decision_start`
    - `date_decision_end`
        - limits of the data. from - to
- __definitions__
    - `families`
        - specification of individual families
        - Python dictionary
        - using [pandas `query` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)
        - these queries are used upon [import_table](#import-table)
    - `name_replacements`
        - replacements of the names which are too long for the database
        - Python dictionary
        - `<part_of_name_that_is_long>` : `<replacement>`
        - as this dictionary is parsed as a text, some characters need escaping.
            - i.e. 
                - `[` must be written as `\[`
                - `.` -> `\.`
                
</details>

# Running the script


In [ ]:
import os
import sys
sys.path.insert(0, "..")
from data_download.data_creator import DataCreator

In folder `config`, we have two configuration files created.  
These need to be modified to access the correct database, and where we can specify correct handling of individual attributes.

Let's see, what config files we can use:

In [ ]:
os.listdir("./config_data_creator")

## Class initialisation (OFFLINE)

Arguments:
- `db_config` - path to [db_config](#db-config)
- `script_config` - path to [script_config](#script-config)
- logger arguments
     - log_name {str} -- name of the logging object 
     - log_filename {str} -- file address where the log should be saved 
     - log_level {int} -- logger level which should be stored (default: logging.DEBUG)
     - log_format {str} -- formatting of the logging entries 
     - handlers {str} -- 'both', 'file', 'stream' - which handlers should be added
     
We initialize the class with configuration files.

In [ ]:
dc = DataCreator(
    db_config="./config_data_creator/sc_data_db_config_template.cfg",
    script_config="./config_data_creator/sc_data_script_config_template.cfg",
    log_level=10,
    download_tnsnames_ora=True
)

## Attribute loading (OFFLINE)
__You are now advised to modify the attributes xlsx - column IMPORT_THIS_ATTRIBUTE__

This `attributes.xlsx` have following columns:  
```
skp_scoring_vector_attribute
text_vector_attr_full_path
flag_array
name_vector_attr_datatype
cnt_cred_with_val
cnt_cred_all
max_dist_vals
cnt_bod1
cnt_bod2
scoring_phase
IMPORT_THIS_ATTRIBUTE
```

Afterwards, you can just run the next attribute. **No need to specify address - it is inside the class**

Arguments:
- excel - address of the excel file (if saved ase xlsx, on some other place than default)
- csv - adress of the csv file (if saved as the csv, on some other place than default)

In [ ]:
dc.create_attributes(attributes_path='attributes.xlsx')

In [ ]:
attributes = dc.load_attributes()
attributes

## Creation of import table

We modify the `attributes` table to create `import_table`.  
We create the table and we get whether column names are too long for further work.


If yes: modify the `"../config/sc_data_script_config_template.cfg"`,  
part `name_replacements` to get correct column names.

In [ ]:
import_table = dc.get_import_table()
import_table

We need to send several columns from the import table to the database.  
Luckily, we have a command prepared for that.

In [ ]:
dc.send_import_table_to_db()

## Temp table creation

We are ready to go.  
First, we create the temp table with vector subset

In [ ]:
dc.create_temp_table_db()

## Creation of tables needed for scorecard development
Now, we create all individual tables specified in `"../config/sc_data_script_config_template.cfg"`

In [ ]:
dc.orchestrate_data_table_creation()

... Tadaaa.

Now you can download these tables from the database, just as before, or you can download them here

## Data Download

In [ ]:
final_data = dc.orchestrate_data_download(
    tables=None,
    limit=None,
    chunksize=1000,
    save_to="C:/Data/"
)

for name, table in final_data.items():
    print(name)
    print(table.head())